In [1]:
import os
from covidanalytics import html

In [2]:
reports_path=os.path.join(os.environ['HOME'],'Documents/code/COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/')
html_path=os.path.join(os.environ['HOME'],'Documents/code/covid-static/pages/')

In [3]:
end_date = '04/07/2020'

In [4]:
usa = {'reports_path': reports_path,
          'html': {'dir': html_path + 'usa.html',
                   'img': 'usa.png',
                   'img_shape': 'wide'
                  },
          'country': 'US',
          'start_date': '01/22/2020',
          'end_date': end_date,
          'title': 'USA - COVID-19'
         }

In [5]:
ca = {'reports_path': reports_path,
          'html': {'dir': html_path + 'california.html',
                   'img': 'california.png',
                   'img_shape': 'tall'
                  },
          'state': 'California',
          'start_date': '01/22/2020',
          'end_date': end_date,
          'title': 'California - COVID-19'
         }

In [6]:
bay = {'reports_path': reports_path,
          'html': {'dir': html_path + 'bay_area.html',
                   'img': 'bay_area.png',
                   'img_shape': 'square'
                  },
          'query': 'Province_State=="California" & Admin2 in ["Alameda","Santa Clara","Santa Cruz","San Francisco","San Mateo","Marin","Contra Costa"]',
          'start_date': '03/22/2020',
          'end_date': end_date,
          'title': 'Bay Area - COVID-19'
         }

In [7]:
md = {'reports_path': reports_path,
          'html': {'dir': html_path + 'maryland.html',
                   'img': 'maryland.png',
                   'img_shape': 'wide'
                  },
          'state': 'Maryland',
          'start_date': '01/22/2020',
          'end_date': end_date,
          'title': 'Maryland - COVID-19'
         }

In [8]:
nj = {'reports_path': reports_path,
          'html': {'dir': html_path + 'new_jersey.html',
                   'img': 'new_jersey.png',
                   'img_shape': 'tall'
                  },
          'state': 'New Jersey',
          'start_date': '01/22/2020',
          'end_date': end_date,
          'title': 'New Jersey - COVID-19'
         }

In [9]:
ny = {'reports_path': reports_path,
          'html': {'dir': html_path + 'new_york.html',
                   'img': 'new_york.png',
                   'img_shape': 'wide'
                  },
          'state': 'New York',
          'start_date': '01/22/2020',
          'end_date': end_date,
          'title': 'New York - COVID-19'
         }

In [10]:
wa = {'reports_path': reports_path,
          'html': {'dir': html_path + 'washington.html',
                   'img': 'washington.png',
                   'img_shape': 'wide'
                  },
          'state': 'Washington',
          'start_date': '01/22/2020',
          'end_date': end_date,
          'title': 'Washington - COVID-19'
         }

In [11]:
ore = {'reports_path': reports_path,
          'html': {'dir': html_path + 'oregon.html',
                   'img': 'oregon.png',
                   'img_shape': 'wide'
                  },
          'state': 'Oregon',
          'start_date': '01/22/2020',
          'end_date': end_date,
          'title': 'Oregon - COVID-19'
         }

In [12]:
me = {'reports_path': reports_path,
          'html': {'dir': html_path + 'maine.html',
                   'img': 'maine.png',
                   'img_shape': 'tall'
                  },
          'state': 'Maine',
          'start_date': '01/22/2020',
          'end_date': end_date,
          'title': 'Maine - COVID-19'
         }

In [13]:
va = {'reports_path': reports_path,
          'html': {'dir': html_path + 'virginia.html',
                   'img': 'virginia.png',
                   'img_shape': 'wide'
                  },
          'state': 'Virginia',
          'start_date': '01/22/2020',
          'end_date': end_date,
          'title': 'Virginia - COVID-19'
         }

In [14]:
mi = {'reports_path': reports_path,
          'html': {'dir': html_path + 'michigan.html',
                   'img': 'michigan.png',
                   'img_shape': 'tall'
                  },
          'state': 'Michigan',
          'start_date': '01/22/2020',
          'end_date': end_date,
          'title': 'Michigan - COVID-19'
         }

In [15]:
locs = [usa,ca,bay,md,nj,wa,ore,me,va,mi,ny]

In [16]:
for l in locs:
    html.gen_covid_html(l)